In [1]:
# pip install finance-datareader

In [9]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr

from sklearn.linear_model import LinearRegression
from tqdm import tqdm

# 주식 종료 가격 예측 경진대회

## 문제 정의
주식 종료 가격 예측 경진대회는 다음 두가지에 대해 실시합니다.  
(1) 2021년 11월 1일 ~ 2021년 11월 5일 : public 점수     
(2) 2021년 11월 29일 ~ 2021년 12월 3일 : private 점수  
따라서 2021년 12월 3일까지를 유효하게 생각해줘야 하겠습니다.  

## 데이터 로딩을 통한 구체적 문제 파악

In [4]:
stock_list = pd.read_csv('C:/Users/USER/Desktop/Data_Practices/dacon_stock_prediction/stock_list.csv')

In [5]:
stock_list

,종목명,종목코드,상장시장
0,삼성전자,5930,KOSPI
1,SK하이닉스,660,KOSPI
2,NAVER,35420,KOSPI
3,카카오,35720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,64260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


우선 코스피, 코스탁 각각 몇개인지를 확인해보자.

In [6]:
stock_list['상장시장'].value_counts()

KOSPI     185
KOSDAQ    185
Name: 상장시장, dtype: int64

코스피 185개, 코스닥 185개에 대해서 분석할 것이다.

종목코드에 대해서 6자리로 채울 것이다.

In [7]:
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))

가령 예시로, 삼성전자를 바탕으로 한번 진행해보자.

In [8]:
start_date = '20210601'
end_date = '20211031'
sample_code = '005930'
stock = fdr.DataReader(sample_code, start = start_date, end = end_date)
stock

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-06-01,80500,81300,80100,80600,14058401,0.001242
2021-06-02,80400,81400,80300,80800,16414644,0.002481
2021-06-03,81300,83000,81100,82800,29546007,0.024752
2021-06-04,82700,82700,81500,82200,18112259,-0.007246
2021-06-07,82700,82800,81600,81900,16496197,-0.003650
...,...,...,...,...,...,...
2021-10-25,69900,70600,69500,70200,10029621,-0.002841
2021-10-26,70600,71500,70400,71100,10528252,0.012821
2021-10-27,71000,71000,70000,70100,10295316,-0.014065


다음과 같이, 시가, 최고치, 최저치, 종가 데이터를 얻을 수 있다.
문제에서 물어보는 것은 종가에 대한 예측이나, 전체적인 흐름에 대해서는 고려해야 할 것이다.

## Baseline _ (Dacon) : 선형회귀 모델

우선 Dacon에서 제공한 선형회귀 모델을 적용해보자.

In [10]:
start_date = '20210104'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (220, 1)


,Date
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08


In [13]:
sample_submission = pd.read_csv('C:/Users/USER/Desktop/Data_Practices/dacon_stock_prediction/sample_submission.csv')

In [14]:
model = LinearRegression()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    #  종가에 대해서만 예측한다.
    data = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    
    x = data.iloc[0:-2].to_numpy() # 2021년 1월 04일 ~ 2021년 10월 22일까지의 데이터로
    y = data.iloc[1:-1].to_numpy() # 2021년 1월 11일 ~ 2021년 10월 29일까지의 데이터를 학습한다.
    y_0 = y[:,0]
    y_1 = y[:,1]
    y_2 = y[:,2]
    y_3 = y[:,3]
    y_4 = y[:,4]

    y_values = [y_0, y_1, y_2, y_3, y_4]
    x_public = data.iloc[-2].to_numpy() # 2021년 11월 1일부터 11월 5일까지의 데이터를 예측할 것이다.
    
    predictions = []
    for y_value in y_values :
        model.fit(x,y_value)
        prediction = model.predict(np.expand_dims(x_public,0))
        predictions.append(prediction[0])
    sample_submission.loc[:,code] = predictions * 2
sample_submission.isna().sum().sum()

100%|██████████| 370/370 [00:46<00:00,  8.02it/s]


0

In [15]:
columns = list(sample_submission.columns[1:])

columns = ['Day'] + [str(x).zfill(6) for x in columns]

sample_submission.columns = columns

sample_submission.to_csv('BASELINE_Linear.csv',index=False)

## Arima의 적용